## Introduction
#### 대부분의 경우 데이터는 여러 파일이나 데이터베이스 또는 분석하기 쉽지 않은 형태로 기록되어 있다. 이 장에서는 데이터를 합치고, 
#### 재배열 할 수 있는 도구들을 알아본다. 

## 계층적 색인 
#### 계층적 색인은 pandas의 중요한 기능인데, 축에 대해 다중(둘 이상) 색인 단계를 지정할 수 있도록 해준다. 
#### 즉, 높은 차원의 데이터를 낮은 차원의 형식으로 다룰수 있도록 해주는 기능이다. 

In [2]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                      [1, 2, 3, 1, 3, 1, 2, 2, 3]])   #Multiindex를 색인으로 하는 Series

data

a  1   -0.657585
   2   -0.883267
   3    0.115917
b  1    0.793523
   3   -0.194407
c  1    1.370835
   2    1.715212
d  2    0.525528
   3    0.923908
dtype: float64

In [2]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [3]:
#계층적으로 색인된 객체는 데이터의 부분집합을 '부분적 색인으로 접근'하는것이 가능하다.
data['b']

1   -0.748116
3    2.291690
dtype: float64

In [4]:
data['b':'c']

b  1   -0.748116
   3    2.291690
c  1    1.220454
   2    0.654093
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1   -0.748116
   3    2.291690
d  2   -0.473314
   3    1.049103
dtype: float64

In [6]:
data.iloc[0:2]

a  1   -0.135140
   2   -1.405528
dtype: float64

In [3]:
data.loc[:,2] #하위 계층의 객체를 선택하는 예

a   -0.883267
c    1.715212
d    0.525528
dtype: float64

#### 계층적 색인은 데이터를 재생성하고 피벗테이블 생성 같은 그룹 기반의 작업을 할때 중요하게 사용된다. 

In [8]:
data

a  1   -0.135140
   2   -1.405528
   3    0.521588
b  1   -0.748116
   3    2.291690
c  1    1.220454
   2    0.654093
d  2   -0.473314
   3    1.049103
dtype: float64

In [4]:
#unstack 메소드를 사용해서 데이터를 재배열
data.unstack() #로우를 컬럼으로 
#data.unstack(level=0)
#data.unstack(level=1)

,1,2,3
a,-0.657585,-0.883267,0.115917
b,0.793523,NaN,-0.194407
c,1.370835,1.715212,NaN
d,NaN,0.525528,0.923908


In [5]:
data.unstack().stack() # 원상복구
#unstack의 반대 작업은 stack 메소드로 수행, 후반부에 stack과 unstack은 한번 더 자세히 언급 됨

a  1   -0.657585
   2   -0.883267
   3    0.115917
b  1    0.793523
   3   -0.194407
c  1    1.370835
   2    1.715212
d  2    0.525528
   3    0.923908
dtype: float64

#### DataFrame은 두 축 모두 계층적 색인을 가질 수 있다. 

In [156]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                    columns = [['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [157]:
frame.index.names = ['key1', 'key2'] #계층적 색인에 이름을 부여하고 출력하기
frame.columns.names = ['state', 'color'] 
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [158]:
frame['Ohio'] #컬럼의 부분집합을 부분적인 색인으로 접근

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [159]:
frame['Colorado'] #컬럼의 부분집합을 부분적인 색인으로 접근

color      Green
key1 key2       
a    1         2
     2         5
b    1         8
     2        11

In [160]:
frame['Green'] #error 발생, Green은 Ohio의 하위 컬럼이름

KeyError: 'Green'

In [161]:
frame['Ohio', 'Green']
#frame['Ohio']['Green']

key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int32

In [162]:
#MultiIndex는 따로 생성한다음 재사용 가능하다.
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                      names = ['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [163]:
mindex = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                      names = ['state', 'color'])
frame_test = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                    columns = mindex)
frame_test

state  Ohio     Colorado
color Green Red    Green
a 1       0   1        2
  2       3   4        5
b 1       6   7        8
  2       9  10       11

In [164]:
frame_test.index.names=['key1', 'key2']
frame_test

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 계층의 순서를 바꾸고 정렬하기 
#### 계층적 색인에서 계층의 순서를 바꾸거나 지정된 계층에 따라 데이터를 정렬해야 하는 경우가 있다. 
#### 'swaplevel'은 넘겨받은 두 계층의 번호나 이름이 뒤바뀐 새로운 객체를 반환한다(하지만 데이터는 변경되지  않는다.). 

In [165]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [166]:
frame.index.nlevels
#frame.columns.nlevels

2

In [167]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [168]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [169]:
frame.sort_index(level=0)
#sort_index를 사용해서 결과가 사전적으로 정렬되도록 만든다.

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [170]:
frame.sort_index(level=1) 
# 다중 인덱스의 두 번째 수준을 기준으로 정렬합니다.
# frame.sort_index(level=1, ascending=True)도 가능
# abab로 바뀜

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [171]:
frame.swaplevel(0,1)
# 첫 번째 수준의 인덱스와 두 번째 수준의 인덱스를 서로 바꿉니다.

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [172]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [173]:
frame.swaplevel(0,1).sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

#### 계층별 요약통계 

In [174]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [175]:
#frame.sum(level='key2') #key2를 기준으로 같은 항목들을 더해준다.
frame.sum(level=1)

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [176]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [177]:
frame.sum(level='color', axis=1) #color를 기준으로 같은 값을 더해준다. Green이 2개이므로 Green을 더해준다. column이므로 axis=1추가

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [42]:
frame.sum(level='state', axis=1)

state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

In [178]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [181]:
#frame.groupby(level="key2").sum()  #DataFrame에서 행이나 열의 합을 계층적으로 구할 수 있다. grupuby()는 10장에서 다시 다룬다.
frame.groupby(level="key1").sum()

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [180]:
frame.groupby(level="color", axis="columns").sum() #위에서 실행한 frame.sum(level='state', axis=1)과 같은 결과

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### DataFrame의 columns 사용하기

In [182]:
frame = pd.DataFrame({'a': range(7), 'b':range(7,0,-1),
                     'c':['one', 'one', 'one', 'two', 'two',
                          'two', 'two'],
                      'd': [0,1,2,0,1,2,3]})
frame                 

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [51]:
frame2 = frame.set_index(['c', 'd']) #set_index함수는 하나 이상의 컬럼을 색인으로 하는 DataFrame을 생성한다.
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [52]:
frame.set_index(['c','d'], drop=False) #컬럼을 명시적으로 남기지 않으면 DataFrame에서 사라진다.

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [53]:
frame2.reset_index() #reset_index는 set_index의 반대 개념으로 원래의 컬럼으로 돌아간다.

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 데이터 합치기
#### pandas 객체에 저장된 데이터는 여러가지 방법으로 합칠 수 있다. 
####     1. 'pandas.merge'는 하나 이상의 키를 기준으로 DataFrames의 row를 합친다. 
####     2. 'pandas.concat'는 하나의 축을 따라 객체를 이어 붙인다. 
####     3. 'combine_first' 메소드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울수 있게 한다. 

### 데이터베이스 스타일로 DataFrame 합치기 
#### 병합(Merge)과 조인(Join) 연산은 관계형 데이터베이스의 핵심이다. 
#### 하나 이상의 키를 사용해서 데이터 집합의 row를 합친다.  
#### pandas의 merge함수를 이용해서 데이터에 적용할 수있다.  

In [2]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],
                   'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [4]:
pd.merge(df1, df2) #어떤 컬럼을 병합할지 명시하지 않으면, merge함수는 중복된 컬럼이름을 키로 사용(이 예에서는 'key')
#key 값이 공통적으로 가지고 있는 항목들만 병합

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [5]:
pd.merge(df1, df2, on='key') # key를 공통으로 해주세요!

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [6]:
#merge는 교집합 결과를 반환한다.
df3 = pd.DataFrame({'lkey': ['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],
                   'data2': range(3)})

df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [7]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [8]:
# 만약 두 객체에 중복된 컬럼이 하나도 없다면 따로 지정해주면 된다.
pd.merge(df3, df4, left_on='lkey', right_on='rkey')
# lkey랑 rkey랑 둘다 공통인것만 나오게된다.

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [9]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [10]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


#### 다대다 병합은 잘 정의되어 있지만 직관적이진 않다. 
#### 다대다 조인은 두 로우의 카르테시안 곱을 반환한다. 

In [11]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [12]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [14]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [13]:
pd.merge(df1, df2, how='right') #오른쪽 테이블에 존재하는 모든 키 조합을 사용한다.
#오른쪽에 해당하는 df2객체의 키값을 기준으로 병합, df1객체의 c는 빠지게 된다.

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


In [66]:
pd.merge(df1, df2, how='left') #왼쪽 테이블에 존재하는 모든 키 조합을 사용한다.
#왼쪽에 해당하는 df1객체의 키값을 기준으로 병합, df2객체의 d는 빠지게 된다.

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [67]:
#how인자로 'left', 'right', 'outer'사용
#left : 왼쪽 테이블에 사용하는 모든 키 조합을 사용한다.
#right : 오른쪽 테이블에 사용하는 모든 키 조합을 사용한다. 
#outer : 양쪽 테이블에 존재하는 모든 키값을 사용
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### 다대다 병합은 잘 정의되어 있지만 직관적이진 않다. 

In [68]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a', 'b'],
                   'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],
                   'data2': range(5)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [69]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [71]:
pd.merge(df1, df2, on='key', how='left') #다대다의 경우, 두 로우의 데카르트곱을 반환, 왼쪽 b 3개, 오른쪽 b 2개 -> 6개의 b 

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [72]:
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [73]:
pd.merge(df1, df2, how='inner') #inner는 양쪽 테이블 모두에 존재하는 키 조합을 사용

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


#### 여러개의 키를 병합하려면 칼람 이름이 담긴 리스트를 넘기면 된다. 

In [100]:
left = pd.DataFrame({'key1':['foo', 'foo', 'bar'],
                    'key2':['one', 'two', 'one'],
                    'lval':[1, 2, 3]})
right = pd.DataFrame({'key1':['foo', 'foo', 'bar', 'bar'],
                     'key2': ['one', 'one', 'one', 'two'],
                     'rval': [4, 5, 6, 7]})



In [101]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [102]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [105]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [78]:
# 병합연산에서 고려해야할 마지막 사항은 겹치는 컬럼 이름에 대한 처리이다.
# left와 right 모두에 공통으로 key2가 들어가 있다, 즉 컬럼 이름이 걑이 겹치게 된다.
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [79]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 색인 병합하기 
#### 병합하려는 키가 DataFrame의 색인인 경우
#### left_index=true 혹은 right_index=true을 지정해서 해당 색인을 병합키로 사용할 수 있다.

In [23]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],
                     'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a','b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [86]:
right1

,group_val
a,3.5
b,7.0


In [88]:
pd.merge(left1, right1, left_on='key', right_index=True)
#pd.merge(left1, right1, left_on='key') #error 발생

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [89]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### 계층 색인된 데이터는 암묵적으로 여러 키를 병합하는것이라 복잡하다.
#### 리스트로 여러개의 컬럼을 지정해서 병합해야 한다.(중복되는 색인값을 다룰때는 how-outer옵션사용)
#### 양쪽에 공통적으로 존재하는 여러개의 색인을 병합하는것도 가능하다. 

In [90]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                     'key2': [2000, 2001, 2002, 2001, 2002],
                     'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                     index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                            'Ohio', 'Ohio'],
                           [2001, 2000, 2000, 2000, 2001, 2002]],
                     columns=['event1', 'event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [91]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [81]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [92]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], 
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


#### 양쪽에 공통적으로 존재하는 여러 개의 색인을 병합하는것도 가능하다.

In [1]:
import pandas as pd
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=['a', 'c', 'e'],
                    columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b','c','d','e'],
                     columns=['Missouri','Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [17]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [18]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)
#pd.merge(left2, right2, how='outer') #error 발생

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### 색인으로 병합할때 DataFrame의 join메소드를 사용하면 편리하다. 
#### join메소드는 컬럼이 겹치지 않으며 완전히 같거나 유사한 색인구조를 가진 여러개의 DataFrame 객체를 병합할때 사용할 수있다.  

In [19]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [20]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [2]:
pd.merge(left2,right2, left_index=True, right_index=True, how='outer')
# outer은 같음.

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [106]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [3]:
left2.join(right2) # 그냥 ace 기준으로 뒤에 right2만 붙이는거임. 

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [4]:
pd.merge(left2,right2, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [24]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [25]:
right1

,group_val
a,3.5
b,7.0


In [26]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [29]:
pd.merge(left1, right1, left_on='key', right_index=True)
#pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


#### 색인 대 색인으로 DataFrame을 병합하려면 join을 사용하면 된다. 하지만 보통 이런 병합은 concat 메소드를 사용한다. 

In [30]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index=['a', 'c', 'e', 'f'],
                      columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [31]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [32]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### 축 따라 이어 붙이기
#### 데이터를 합치는 또 다른 방법으로 이어붙이기(concatenation, binding or stacking이라고도 한다.)가 있다. 
#### NumPy에서는 'concatenate' 함수를 제공한다.

In [33]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [34]:
np.concatenate([arr,arr],axis=1) # axis=1은 가로로이어붙이고

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [35]:
np.concatenate([arr,arr])# axis=0은 밑에 붙임.

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

#### Series나 DataFrame 같은 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 붙일수있도록 되어 있다. 
#### 이때 다음 사항을 고려해야 한다. 
####     1. 만약 연결하려는 두 객체의 색인이 서로 다르다면 결과는 그 색인의 교집합이어야 하는가? 아니면 합집합이어야 하는가?
####     2. 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할수있어야 하는가? 
####     3. 어떤 축으로 연결할것인지 고려해야 하는가? 많은경우 DataFrame의 기본 정수 라벨이 가장먼저 무시된다. 

In [36]:
s1 = pd.Series([0,1], index=['a', 'b'])
s2 = pd.Series([2,3,4], index=['c', 'd', 'e'])
s3 = pd.Series([5,6], index=['f', 'g'])

In [37]:
s1

a    0
b    1
dtype: int64

In [38]:
s2

c    2
d    3
e    4
dtype: int64

In [39]:
s3

f    5
g    6
dtype: int64

In [40]:
pd.concat([s1, s2, s3]) # 세객체를 묶어서 concat 함수에 전달하면 값과 색인을 연결해준다.

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [41]:
pd.concat([s1,s2,s3], axis=1) # 여기서 axis는 dataframe 만들기용

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### 겹치는 축이 없기 때문에 외부 조건으로 정렬된 합집합을 얻었지만, join=inner를 넘겨서 교집합을 구할수도 있다.

In [42]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [43]:
pd.concat([s1, s4], axis=1) # 여기서 axis는 dataframe 만들기용

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [44]:
pd.concat([s1, s4], axis=1, join='inner') #f와 g는 join-inner 옵션으로 사라진다. 

,0,1
a,0,0
b,1,1


In [45]:
pd.concat([s1, s4], axis=1).reindex(['a', 'c', 'b', 'e'])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


#### Series를 이어붙이기 전에 개별 Series를 구분할수 없는 문제가 생기는데, 이어붙인 축에 대해 계층적 색인을 생성하여 식별이 가능하도록 할수있다. 계층적 색인을 사용할련 keys 인자를 사용한다. 

In [132]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three']) #계층적 색인 사용
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [133]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [120]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']) #Series를 axis=1로 병합하면 DataFrame의 컬럼 제목이 된다.

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [134]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a', 'b', 'c'],
                  columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=['a', 'c'],
                  columns=['three', 'four'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [135]:
df2

,three,four
a,5,6
c,7,8


In [136]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [137]:
pd.concat({'level1':df1, 'level2': df2}, axis=1) #리스트 대신 객체의 사전을 넘기면 사전의 키가 keys옵션으로 사용된다. 

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [138]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
         names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

#### 마지막으로 DataFrame의 로우 색인이 분석에 필요한 데이터를 포함하고 있지 않은경우는 어떻게 할것인가? 

In [139]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2,3), columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,-1.960212,-0.862624,0.098485,0.453295
1,0.489290,0.487781,0.510642,0.253587
2,0.047346,-1.999354,0.500162,0.373158


In [140]:
df2

,b,d,a
0,0.504086,0.273480,-0.795432
1,0.807083,0.008288,1.015091


In [141]:
pd.concat([df1, df2], ignore_index=True)
#pd.concat([df1, df2])

,a,b,c,d
0,-1.960212,-0.862624,0.098485,0.453295
1,0.489290,0.487781,0.510642,0.253587
2,0.047346,-1.999354,0.500162,0.373158
0,-0.795432,0.504086,NaN,0.273480
1,1.015091,0.807083,NaN,0.008288


### 겹치는 데이터 합치기
#### 데이터를 합칠때 병합이나 이어붙이기로 불가능할 경우가 있다.  두 데이터셑의 색인이 일부 겹치거나 전체가 겹치는 경우

In [46]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
             index=['f','e','d','c','b','a'])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
             index=['a','b','c','d','e','f'])

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [47]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [48]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [49]:
a.combine_first(b) #위와 동일한 기능에 정렬기능까지 제공한다.

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [146]:
b.combine_first(a) #위와 동일한 기능에 정렬기능까지 제공한다.

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [50]:
df1 = pd.DataFrame({'a':[1., np.nan, 5., np.nan],
                   'b':[np.nan, 2., np.nan, 6.],
                   'c':range(2,18,4)})
df2 = pd.DataFrame({'a':[5., 4., np.nan, 3., 7.],
                   'b':[np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [51]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [52]:
df1.combine_first(df2) 
#DataFrame에서 combine_first메소드는 같은 동작을 한다.  호출하는 객체에서 누락된 데이터를 인자로 넘긴 객체의 값으로 채운다.

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [53]:
df2.combine_first(df1) 

,a,b,c
0,5.0,NaN,2.0
1,4.0,3.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 재형성과 피벗(Reshaping and Pivoting)
#### 표 형식의 데이터를 재배치하는 다양한 기본연산이 존재한다.
#### 이러한 연산을 재형성 또는 피벗 연산이라고 한다, 

### 계층적 색인으로 재형성하기 
#### 계층적 색인은 DataFrame의 데이터를 재배치하는 다음고 같은 방식을 제공한다. 
####     1. stack - 데이터의 열을 행으로 피벗(또는 회전) 시킨다. 
####     2. unstack - 행을 열로 피벗 시킨다.

In [54]:
import pandas as pd
import numpy as np
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns = pd.Index(['one', 'two', 'three'],
                                     name = 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [55]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [36]:
result.unstack() #기본적으로 가장 안쪽에 있는 레벨부터 끄집어낸다.

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### 기본적으로 가장 안쪽에 있는 레벨부터 끄집어내는데(stack도 마찬가지), 레벨숫자나 이름을 전달해서 끄집어낼 단계를 지정할 수있다. 
#### 해당 레벨에 있는 모든 값이 하위그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길수 있다. 

In [37]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [39]:
result.unstack(1) #0에 해당하는 state 색인값이 컬럼으로 간다.

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [40]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [122]:
s1 = pd.Series([0,1,2,3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4,5,6], index=['c','d','e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [123]:
data2.unstack() #해당 레벨에 있는 모든 값이 하위그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길수 있다.

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


#### stack 메서드는 누락된 데이터를 자동으로 걸러내기 때문에 연산을 쉽게 원상복구할 수있다. 

In [125]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [126]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [127]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [56]:
df = pd.DataFrame({'left': result, 'right': result+5},
                 columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [57]:
df.stack()

state     number  side 
Ohio      one     left      0
                  right     5
          two     left      1
                  right     6
          three   left      2
                  right     7
Colorado  one     left      3
                  right     8
          two     left      4
                  right     9
          three   left      5
                  right    10
dtype: int32

In [129]:
df.unstack('state') #DataFrame을 unstack할때 레벨은 결과에서 가장 낮은 단계가 된다.

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [130]:
df.unstack('state').stack('side') #stack을 호출할때 쌓을 축의 이름을 지정할 수있다.

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 긴(Long) 형식에서 넓은(Wide) 형식으로 피벗하기
#### 시계열 예제

In [77]:
data = pd.read_csv("macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959.0,1.0,2710.349,0.00,5.8
1,1959.0,2.0,2778.801,2.34,5.1
2,1959.0,3.0,2775.488,2.74,5.3
3,1959.0,4.0,2785.204,0.27,5.6
4,1960.0,1.0,2847.699,2.31,5.2


In [78]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [79]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [80]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [81]:
long_data = (data.stack().reset_index())
long_data[:10]

,date,item,0
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [82]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


#### pivot메소드의 처음 두인자는 로우와 컬럼 색인으로 사용될 컬럼이름이고 마지막 인자는 DataFrame에 채워넣을 값을 담고있는 컬럼 이름이다. 

In [83]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [84]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.159521
1,1959-01-01,infl,0.000,0.665714
2,1959-01-01,unemp,5.800,0.930023
3,1959-04-01,realgdp,2778.801,0.250614
4,1959-04-01,infl,2.340,-1.168458
5,1959-04-01,unemp,5.100,0.157755
6,1959-07-01,realgdp,2775.488,0.418901
7,1959-07-01,infl,2.740,0.987673
8,1959-07-01,unemp,5.300,-0.158869
9,1959-10-01,realgdp,2785.204,0.450696


In [86]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()
#pivoted["value"].head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.665714 -0.159521  0.930023
1959-04-01  2.34  2778.801   5.1 -1.168458  0.250614  0.157755
1959-07-01  2.74  2775.488   5.3  0.987673  0.418901 -0.158869
1959-10-01  0.27  2785.204   5.6 -2.403409  0.450696 -0.630453
1960-01-01  2.31  2847.699   5.2 -0.006014  1.931594 -1.452406

In [90]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.665714 -0.159521  0.930023
1959-04-01  2.34  2778.801   5.1 -1.168458  0.250614  0.157755
1959-07-01  2.74  2775.488   5.3  0.987673  0.418901 -0.158869
1959-10-01  0.27  2785.204   5.6 -2.403409  0.450696 -0.630453
1960-01-01  2.31  2847.699   5.2 -0.006014  1.931594 -1.452406

### 넓은(Wide)형식에서 긴(Long)형식으로 피벗하기 
#### 'pivot'과 반대되는 연은 'pandas.melt'이다 .
#### 하나의 컬렄을 여러개의 새로운 DataFrame으로 생성하기보다는 여러 컬럼을 하나로 합병하고 DataFrame을 입력보다 긴 형태로 만들어 낸다. 

In [93]:
df =pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                 'A': [1,2,3],
                 'B': [4,5,6],
                 'C': [7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [94]:
melted = pd.melt(df, ['key']) #melt를 사용할때에는 반드시 어떤 컬럼을 구분자로 사용할 것인지 지정해야 한다. 여기서는 'key'사용
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


#### We can use pivot to get the original layout back. But because it creates an index from the columns used as row labels, we may have to use 'reset_index' to move it back into a column.
#### You can be explicit in which columns can be indicators and which can be values using the 'id_vars' and 'value_vars' parameters respectively.
#### We can use pandas.melt without group identifiers too.

In [95]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [96]:
reshaped.reset_index() #pivot은 로우를 컬럼색인으로 사용하므로 reset_index를 이용해서 데이터를 다시 컬럼으로 되돌린디.

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [40]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [41]:
pd.melt(df, id_vars=['key'], value_vars=['A','B']) #데이터값으로 사용할 컬럼들의 집합을 지정할 수 있다.

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [42]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [43]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
